In [1]:
import sys
sys.path.insert(0, '/home/ldoyle/packages')
import h5py
import numpy as np
from tqdm import tqdm
import matplotlib
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import pysindy as ps
from lr_ed import localreg
from datetime import datetime
from scipy.fft import fft, fftfreq,rfft2

import scipy.signal as signal
from obspy.signal import filter as obsfilt
import glob


In [2]:
WEAK = False

x_len = 5000
t_len = 6000

xs = np.arange(x_len)
dt = 1

library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]  

if WEAK:
    X, T = np.meshgrid(xs, np.arange(t_len))
    XT = np.asarray([X, T]).T

    pde_lib = ps.WeakPDELibrary(
        library_functions=library_functions,
        function_names=library_function_names,
        derivative_order=4,
        spatiotemporal_grid=XT,
        is_uniform=True,
        K=1000,
    )
else:
    
    pde_lib = ps.PDELibrary(
        library_functions=library_functions,
        function_names=library_function_names,
        derivative_order=4,
        spatial_grid=xs,
        include_bias=True,
        is_uniform=True,
    )
    

In [3]:
def printEnsemble(model, cut_off = 1e-3, median= False):
    if median:
        coefs = np.median(model.coef_list, axis=0)[0,:]
    else:
        coefs = np.mean(model.coef_list, axis=0)[0,:]
    above_cut_off = np.argwhere(np.abs(coefs)>cut_off).flatten()
    features =  model.get_feature_names()
    equation_str= "(x0)' = "
    for count, val in enumerate(above_cut_off):
        if count >0:
            equation_str+=" + "
        equation_str+= "{0:.4f}".format(coefs[val])
        equation_str+= " "+features[val]
        
    print(equation_str)

In [4]:
def saveEnsemble(model,fileStr,thresh):
    np.save("/home/ldoyle/notebooks/ensembleSINDyArrays_30_run2/"+fileStr+"_thresh_"+"{:.2E}".format(thresh), model.coef_list)
#     print(fileStr+"_thresh_"+"{:.2E}".format(thresh))

In [5]:
def SINDyImplementEnsemble(dataset, fileStr,thresh, alph=0.001, n_models=30, time_steps=400, stlsq_max =100,median=True):
    optimizer = ps.STLSQ(threshold=thresh, alpha=alph, max_iter=stlsq_max)
#     optimizer = ps.EnsembleOptimizer( opt=ps.STLSQ(threshold=thresh, alpha=alph, max_iter=stlsq_max),bagging=True,  n_models = n_models,n_subset =time_steps)  
    model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer, 
                     differentiation_method= ps.differentiation.SmoothedFiniteDifference())
    model.fit(dataset, t=dt,ensemble=True, n_models=n_models, n_subset=time_steps)
    print("Ensemble STLSQ with Threshold "+str(thresh))
    printEnsemble(model,median=median)
    saveEnsemble(model,fileStr,thresh)

## Unfiltered

### Dataset 1

In [6]:
path = "/data/data2/south-data-ejm/hdd/South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T16_09_15-0700/"

fileStr = "2021-11-01T2311_unfiltered"

init = 11
final= 16

time_subsample = 10
full_dat = np.zeros((5000,(12000//time_subsample)*(final-init)))

k = 0
for i in np.arange(init,final):
    file = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T23"+str(i)+"14Z.h5"
    f = h5py.File(path+file, 'r')
    data = f['Acquisition']['Raw[0]']['RawData'][:, :].astype('int64')
#     timestamp = f['Acquisition']['Raw[0]']['RawDataTime'][:] / 1000000
    f.close()
    
    full_dat[:,k*(12000//time_subsample):(k+1)*(12000//time_subsample)] = data[7500:12500,::time_subsample]
    k+=1
    
working_dat = full_dat/np.std(full_dat)


In [7]:
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [9]:
thresh = 1.e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6390 x0_1 + 0.3524 x0_111


In [10]:
thresh = 1.e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6270 x0_1 + 0.3432 x0_111 + 0.0055 x0x0x0_1 + 0.0054 x0x0x0_111


In [11]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6393 x0_1 + 0.3527 x0_111 + 0.0141 x0x0_1 + -0.0075 x0x0_11


In [12]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6421 x0_1 + 0.3536 x0_111 + 0.0109 x0x0_1 + 0.0094 x0x0x0_1 + -0.0104 x0x0_11 + 0.0091 x0x0x0_111 + -0.0056 x0x0_1111


In [13]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = -0.0017 1 + -0.0014 x0 + 0.6404 x0_1 + 0.0050 x0_11 + 0.3477 x0_111 + 0.0036 x0_1111 + 0.0101 x0x0_1 + 0.0098 x0x0x0_1 + 0.0010 x0x0_11 + -0.0013 x0x0x0_11 + 0.0046 x0x0_111 + 0.0070 x0x0x0_111


### Dataset 2

In [14]:
time_subsample = 10
full_dat = np.zeros((5000,6000))
fileStr = "2021-11-01T2321_unfiltered"


k = 0
for i in np.arange(21,26):
    file = "South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T23"+str(i)+"14Z.h5"
    f = h5py.File(path+file, 'r')
    data = f['Acquisition']['Raw[0]']['RawData'][:, :].astype('int64')
#     timestamp = f['Acquisition']['Raw[0]']['RawDataTime'][:] / 1000000
    f.close()
    
    full_dat[:,k*(12000//time_subsample):(k+1)*(12000//time_subsample)] = data[7500:12500,::time_subsample]
    k+=1
    


In [15]:
working_dat = full_dat/np.std(full_dat)#*10**6
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [16]:
thresh = 1.e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6725 x0_1 + 0.3814 x0_111


In [17]:
thresh = 1.e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6620 x0_1 + 0.3770 x0_111 + -0.0130 x0x0_1 + -0.0148 x0x0_111


In [18]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6555 x0_1 + -0.0077 x0_11 + 0.3747 x0_111 + 0.0082 x0x0x0_1 + -0.0155 x0x0_11 + -0.0147 x0x0_111 + 0.0103 x0x0x0_111 + -0.0026 x0x0_1111


In [19]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6761 x0_1 + -0.0163 x0_11 + 0.3881 x0_111 + -0.0049 x0_1111 + 0.0029 x0x0x0_1 + 0.0038 x0x0_11 + -0.0070 x0x0_111


In [20]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.0025 1 + -0.0011 x0x0 + 0.6690 x0_1 + -0.0084 x0_11 + 0.3837 x0_111 + -0.0025 x0x0_1 + 0.0086 x0x0x0_1 + 0.0049 x0x0x0_11 + -0.0108 x0x0_111 + 0.0096 x0x0x0_111 + 0.0030 x0x0x0_1111


## Drift Removal Load

### Dataset 1

In [21]:
fileStr = "2021-11-01T2311_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [22]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6843 x0_1 + 0.3126 x0_111


In [23]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.6883 x0_1 + 0.3240 x0_111


In [24]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6961 x0_1 + 0.3310 x0_111


In [25]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6983 x0_1 + 0.3322 x0_111 + -0.0076 x0x0x0_1 + 0.0069 x0x0_111


In [26]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6980 x0_1 + -0.0061 x0_11 + 0.3263 x0_111 + -0.0031 x0_1111 + 0.0042 x0x0_1 + -0.0088 x0x0x0_1 + 0.0072 x0x0_11 + 0.0043 x0x0x0_11 + 0.0013 x0x0_111 + -0.0073 x0x0x0_111 + 0.0019 x0x0x0_1111


### Dataset 2

In [27]:
fileStr = "2021-11-01T2321_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [28]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6755 x0_1 + 0.3146 x0_111


In [29]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.6663 x0_1 + 0.3112 x0_111


In [30]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6801 x0_1 + 0.3134 x0_111


In [31]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6843 x0_1 + 0.3217 x0_111 + 0.0032 x0x0_1


In [32]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.6802 x0_1 + 0.0060 x0_11 + 0.3139 x0_111 + 0.0038 x0_1111 + 0.0039 x0x0_1 + -0.0071 x0x0x0_1 + -0.0023 x0x0_11 + -0.0014 x0x0_111 + -0.0038 x0x0x0_111


### Dataset 3

In [33]:
fileStr = "2021-11-02T0511_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [34]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6875 x0_1 + 0.3255 x0_111


In [35]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.6875 x0_1 + 0.3222 x0_111


In [36]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6905 x0_1 + 0.3338 x0_111


In [37]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6929 x0_1 + 0.3222 x0_111 + 0.0084 x0x0_1 + -0.0074 x0x0x0_1


In [38]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.7059 x0_1 + -0.0128 x0_11 + 0.3263 x0_111 + -0.0046 x0_1111 + 0.0119 x0x0_1 + -0.0025 x0x0x0_1 + 0.0101 x0x0_11 + 0.0088 x0x0_111 + 0.0029 x0x0_1111


### Dataset 4

In [39]:
fileStr = "2021-11-02T2311_normal"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")[:,::10]
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [40]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.6110 x0_1 + 0.2948 x0_111


In [41]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.6151 x0_1 + 0.2899 x0_111


In [42]:
thresh = 5e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.005
(x0)' = 0.6120 x0_1 + 0.2863 x0_111


In [43]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.6325 x0_1 + 0.0025 x0_11 + 0.2945 x0_111 + -0.0069 x0x0_1 + -0.0097 x0x0x0_1 + 0.0060 x0x0_11 + 0.0033 x0x0x0_11 + 0.0018 x0x0_1111


In [44]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = -0.0016 1 + 0.6144 x0_1 + 0.0129 x0_11 + 0.2882 x0_111 + 0.0028 x0_1111 + -0.0027 x0x0_1 + -0.0075 x0x0_11 + -0.0037 x0x0_111 + -0.0013 x0x0x0_111


## Multifilt

### Dataset 1

In [14]:
fileStr = "2021-11-01T2311_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [7]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9960 x0_1


In [8]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9969 x0_1 + 0.0848 x0_111 + 0.0621 x0x0x0_111


In [9]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9968 x0_1 + 0.0841 x0_111 + 0.0627 x0x0x0_111


In [15]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9969 x0_1 + 0.0858 x0_111 + 0.0627 x0x0x0_111


In [16]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9954 x0_1 + 0.0832 x0_111 + 0.0017 x0x0x0_1 + 0.0685 x0x0x0_111


In [17]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0005
(x0)' = 0.9952 x0_1 + 0.0020 x0_11 + 0.0865 x0_111 + 0.0019 x0x0x0_1 + -0.0012 x0x0_111 + 0.0675 x0x0x0_111


### Dataset 2

In [18]:
fileStr = "2021-11-01T2321_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [19]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9963 x0_1


In [20]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9971 x0_1 + 0.1000 x0_111 + 0.0611 x0x0x0_111


In [21]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9973 x0_1 + 0.0968 x0_111 + 0.0670 x0x0x0_111


In [22]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9971 x0_1 + 0.0945 x0_111 + 0.0641 x0x0x0_111


In [23]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9955 x0_1 + 0.0965 x0_111 + 0.0016 x0x0x0_1 + 0.0703 x0x0x0_111


In [24]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0005
(x0)' = 0.9959 x0_1 + 0.0985 x0_111 + 0.0014 x0x0x0_1 + 0.0020 x0x0_111 + 0.0689 x0x0x0_111


### Dataset 3

In [37]:
fileStr = "2021-11-02T0511_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [26]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9967 x0_1


In [27]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9971 x0_1 + 0.1031 x0_111 + 0.0561 x0x0x0_111


In [28]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9973 x0_1 + 0.1023 x0_111 + 0.0550 x0x0x0_111


In [29]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9974 x0_1 + 0.1045 x0_111 + 0.0562 x0x0x0_111


In [30]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9964 x0_1 + 0.1052 x0_111 + 0.0012 x0x0x0_1 + 0.0650 x0x0x0_111


In [38]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0005
(x0)' = 0.9957 x0_1 + 0.1034 x0_111 + 0.0013 x0x0x0_1 + 0.0657 x0x0x0_111


### Dataset 4

In [6]:
fileStr = "2021-11-02T2311_smoothed_rad_4"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_4_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [7]:
thresh = 1e-1
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.1
(x0)' = 0.9979 x0_1 + 0.2165 x0_111


In [8]:
thresh = 5e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.05
(x0)' = 0.9983 x0_1 + 0.1853 x0_111 + 0.0612 x0x0x0_111


In [9]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9980 x0_1 + 0.1815 x0_111 + 0.0624 x0x0x0_111


In [ ]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

In [ ]:
thresh = 5e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

## Multifilt (radius 10 smooth)

### Dataset 1

In [6]:
fileStr = "2021-11-01T2311_smoothed_rad_10"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [7]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9975 x0_1


In [8]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9976 x0_1


In [9]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9962 x0_1 + -0.0011 x0_111 + 0.0016 x0x0x0_1


In [10]:
thresh = 1e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0001
(x0)' = 0.9961 x0_1 + -0.0011 x0_111 + 0.0016 x0x0x0_1


In [11]:
thresh = 1e-5
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 1e-05
(x0)' = 0.9960 x0_1 + -0.0011 x0_111 + 0.0017 x0x0x0_1


### Dataset 2

In [12]:
fileStr = "2021-11-01T2321_smoothed_rad_10"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-01T232114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [13]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9977 x0_1


In [14]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9978 x0_1


In [15]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9965 x0_1 + -0.0014 x0_111 + 0.0012 x0x0x0_1


In [16]:
thresh = 1e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0001
(x0)' = 0.9965 x0_1 + -0.0014 x0_111 + 0.0012 x0x0x0_1


In [17]:
thresh = 1e-5
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 1e-05
(x0)' = 0.9965 x0_1 + -0.0013 x0_111 + 0.0012 x0x0x0_1


### Dataset 3

In [18]:
fileStr = "2021-11-02T0511_smoothed_rad_10"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T051114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [19]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9979 x0_1


In [20]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9979 x0_1


In [21]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9978 x0_1 + -0.0014 x0_111


In [22]:
thresh = 1e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0001
(x0)' = 0.9968 x0_1 + -0.0014 x0_111


In [23]:
thresh = 1e-5
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 1e-05
(x0)' = 0.9968 x0_1 + -0.0014 x0_111 + 0.0011 x0x0x0_1


### Dataset 4

In [24]:
fileStr = "2021-11-02T2311_smoothed_rad_10"

working_dat = np.load("/home/ldoyle/notebooks/channel_drift_5_min_clips/rad_10_smoothed_decimated_normalized_5_min_start_South-C1-LR-95km-P1kHz-GL50m-SP2m-FS200Hz_2021-11-02T231114Z.npy")
scaler_val = np.std(working_dat)
working_dat = working_dat/scaler_val
working_dat = working_dat.reshape(working_dat.shape[0],working_dat.shape[1],1)


In [25]:
thresh = 1e-2
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.01
(x0)' = 0.9985 x0_1


In [26]:
thresh = 3e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.003
(x0)' = 0.9985 x0_1


In [27]:
thresh = 1e-3
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.001
(x0)' = 0.9984 x0_1 + -0.0021 x0_111


In [28]:
thresh = 1e-4
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 0.0001
(x0)' = 0.9977 x0_1 + -0.0020 x0_111


In [29]:
thresh = 1e-5
SINDyImplementEnsemble(working_dat, fileStr,thresh)

Ensemble STLSQ with Threshold 1e-05
(x0)' = 0.9977 x0_1 + -0.0020 x0_111
